In [ ]:
%env PYTHONX=-Xfrozen_modules=off

<!--parameters-->
- intenstity: 1.0

In [ ]:
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
import numpy as np
import pandas as pd
from collections import Counter
import os
import time
import matplotlib.pyplot as plt
import re
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap

: 

In [ ]:
def filter_snps(input_file, output_file):
    # Loading the Alignment
    alignment = AlignIO.read(input_file, "fasta")

    # Converting SeqRecord objects to strings
    sequences = [str(record.seq) for record in alignment]

    # Filtering the snps
    variable_sites = [i for i in range(alignment.get_alignment_length()) if len(set(sequences[j][i] for j in range(len(sequences)))) > 1]

    # Positions of variable sites in the original sequence
    positions = [position + 1 for position in variable_sites]

    # Create new SeqRecord objects with only variable sites
    variable_records = [
        SeqRecord(Seq(''.join(sequences[j][i] for i in variable_sites)), id=record.id)
        for j, record in enumerate(alignment)
    ]

    # Create a new alignment with the SeqRecord objects
    variable_alignment = MultipleSeqAlignment(variable_records)

    # Saving the new alignment to the output file
    AlignIO.write(variable_alignment, output_file, 'fasta')
    return positions

input_file_path = f"output/output_bottleneck/fasta/bottleneck_{intensity}.fasta"
output_file_path = f"output/output_bottleneck/snps/bottleneck_{intensity}.fasta"

# Calling the function to filter variable sites
positions_of_snps = filter_snps(input_file_path, output_file_path)



In [ ]:
sequences = list(SeqIO.parse(input_file_path, 'fasta'))
i=1
df = pd.DataFrame(columns = range(1,len(sequences[1].seq)+1),index=range(1,len(sequences)+1))
for i ,sequence in enumerate(sequences):
    for l, base in enumerate(sequence.seq):  
        df.at[i + 1, l + 1] = base

def most_common_base(column):
    counter = Counter(column)
    return counter.most_common(1)[0][0]

most_frequent_bases = df.apply(most_common_base)
most_frequent_sequence = Seq(''.join(most_frequent_bases))
record = SeqRecord(most_frequent_sequence, id="most_frequent_sequence", description="Most frequent base at each position")
output_file_anc = f"output/output_bottleneck/ancestral_sequence/ancestral_{intensity}.fasta"
SeqIO.write(record,output_file_anc,'fasta')

In [ ]:
def encode_base(base, ancestral_base):
    if base==ancestral_base:
        return 0
    else:
        base_encoding = {'A': 1, 'T': 2, 'C': 3, 'G': 4 , 'N':5}
        return base_encoding.get(base)

def compare_sequences(ancestral_file, aligned_file, output_file,window_size = 200, step_size=100):
    ancestral_seq = str(SeqIO.read(ancestral_file, 'fasta').seq)
    aligned_seqs = list(SeqIO.parse(aligned_file, 'fasta'))

    
    matrix = np.zeros((len(aligned_seqs),( int((len(ancestral_seq) - window_size) / step_size) + 1)))
    counter = 0
    for i, aligned_seq_record in enumerate(aligned_seqs):
        pos=0
        for j in range(0, len(ancestral_seq) - window_size + 1, step_size):
            ancestral_window = ancestral_seq[j:(j + window_size)]
            aligned_window = str(aligned_seq_record.seq[j:(j + window_size)])
            # Compare each position in the window
            for k, ancestral_pos in enumerate(ancestral_window):
                if ancestral_pos != aligned_window[k]:
                    counter += 1
            matrix[i,pos]=counter
            counter=0
            pos=pos+1
            #0 if aligned_seq_record.seq[j] != ancestral_pos else 1
    
    

    similarity_scores = [sum(row) for row in matrix]

    # Sort sequences based on similarity scores
    sorted_indices = np.argsort(similarity_scores)
    matrix = matrix[sorted_indices, :]

    print("Length of aligned_seqs:", len(aligned_seqs))
    print("Sorted indices:", sorted_indices)
    print(matrix.shape)

    aligned_seqs = [aligned_seqs[idx] for idx in sorted_indices]

    # Save the matrix and reordered seq_ids to the output file
    np.savetxt(output_file, matrix, fmt='%d', delimiter='\t', header='\t'.join(map(str, range(( int((len(ancestral_seq) - window_size) / step_size) + 1)))), comments='')
    return matrix

# Example usage
output_file_matrix = f"output/output_bottleneck/matrix/matrix_{intensity}.txt"

matrix = compare_sequences(ancestral_file=output_file_anc,aligned_file=input_file_path, output_file=output_file_matrix)


output_image = f"output/output_bottleneck/images/image_{intensity}.png"
square_size = 1.0


plt.imshow(matrix, cmap="Greys", aspect='equal', extent=[0, matrix.shape[1], 0, matrix.shape[0]])
plt.axis('off')


# Save the image
plt.savefig(output_image, bbox_inches='tight',dpi=1200,pad_inches=0)
plt.close()


meta_file = "output/output_bottleneck/meta_data/meta_images.tab"
with open(meta_file, "a") as file:
    file.write(f"{intensity}\t{output_file_path}\t{output_file_matrix}\t{output_file_anc}\t{output_image}\n")

In [ ]:
output_file_h1_format=f"output/output_bottleneck/h_format/h_{intensity}.csv"

records = list(SeqIO.parse(output_file_path, 'fasta'))
with open(output_file_h1_format, 'w') as outfile:
    i=0
    
    for k in range(len(records[0].seq)):
        # Extract genotypes from all samples at the current position
        genotypes = ','.join(record.seq[k] for record in records)

        # Write the converted line to the output file
        outfile.write(f"{positions_of_snps[i]},{genotypes}\n")
        i=i+1